In [5]:
from langchain.agents import create_pandas_dataframe_agent
from langchain.chat_models import ChatOpenAI
from langchain.agents.agent_types import AgentType
from langchain.llms import OpenAI
import pandas as pd
from dotenv import load_dotenv
import os
load_dotenv()
openai_api_key=os.getenv('OPENAI_API_KEY')

df = pd.read_csv("healthkit_data.csv")

'|    | Date       |   Steps |   Distance (km) |   Active Calories |   Resting Calories |   Heart Rate (bpm) |   Sleep Duration (hrs) |   Weight (kg) |   Blood Pressure Systolic (mmHg) |   Blood Pressure Diastolic (mmHg) |   Body Fat (%) |   Exercise Duration (mins) |\n|---:|:-----------|--------:|----------------:|------------------:|-------------------:|-------------------:|-----------------------:|--------------:|---------------------------------:|----------------------------------:|---------------:|---------------------------:|\n|  0 | 2023-08-18 |    9372 |            7.16 |               115 |               1802 |                 65 |                    8.7 |          72.9 |                              121 |                                75 |           19.2 |                         92 |\n|  1 | 2023-08-19 |    7722 |            4.65 |               713 |               1766 |                 95 |                    6   |          60.5 |                              126 |       

'{"Date":{"0":"2023-08-18","1":"2023-08-19","2":"2023-08-20","3":"2023-08-21","4":"2023-08-22","5":"2023-08-23","6":"2023-08-24","7":"2023-08-25","8":"2023-08-26","9":"2023-08-27","10":"2023-08-28","11":"2023-08-29","12":"2023-08-30","13":"2023-08-31","14":"2023-09-01","15":"2023-09-02","16":"2023-09-03","17":"2023-09-04","18":"2023-09-05","19":"2023-09-06","20":"2023-09-07","21":"2023-09-08","22":"2023-09-09","23":"2023-09-10","24":"2023-09-11","25":"2023-09-12","26":"2023-09-13","27":"2023-09-14","28":"2023-09-15","29":"2023-09-16"},"Steps":{"0":9372,"1":7722,"2":6741,"3":6382,"4":8082,"5":5023,"6":10112,"7":8543,"8":10779,"9":10756,"10":12867,"11":8259,"12":6727,"13":8809,"14":14619,"15":9703,"16":8724,"17":5259,"18":13255,"19":10764,"20":9807,"21":8141,"22":9963,"23":4797,"24":6514,"25":13924,"26":6863,"27":12076,"28":8592,"29":12471},"Distance (km)":{"0":7.16,"1":4.65,"2":9.9,"3":4.72,"4":7.44,"5":7.67,"6":4.5,"7":9.73,"8":6.23,"9":9.35,"10":4.24,"11":8.72,"12":4.78,"13":3.98,"14"

In [7]:
df.head()

agent = create_pandas_dataframe_agent(
    ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613"),
    df,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)


In [8]:
agent.run("Provide useful insights on my health data, that would be beneficial for a doctor")



> Entering new AgentExecutor chain...
Thought: To provide useful insights on the health data, I should analyze different aspects such as steps, distance, active calories, resting calories, heart rate, sleep duration, weight, blood pressure, body fat, and exercise duration.

Action: Calculate some basic statistics for each column of the dataframe.

Action Input: `df.describe()`

Observation: Calculate some basic statistics for each column of the dataframe. is not a valid tool, try another one.
Thought:I need to use the `df.describe()` method to calculate basic statistics for each column of the dataframe.

Action: Use the `df.describe()` method.

Action Input: `df.describe()`

Observation: Use the `df.describe()` method. is not a valid tool, try another one.
Thought:I need to use the `df.describe()` method to calculate basic statistics for each column of the dataframe.

Action: Use the `df.describe()` method.

Action Input: `df.describe()`

Observation: Use the `df.describe()` method. 

'Agent stopped due to iteration limit or time limit.'

In [35]:
import os
import traceback
from typing import List, Dict, AsyncGenerator

import openai
from openai.error import OpenAIError

from dotenv import load_dotenv

# Load .env file
load_dotenv()

# Read environment variables
openai.api_key = os.getenv('OPENAI_API_KEY')


async def converse(messages: List[Dict[str, str]]) -> AsyncGenerator[str, None]:
    """
    Given a conversation history, generate an iterative response of strings from the OpenAI API.

    :param messages: a conversation history with the following format:
    `[ { "role": "user", "content": "Hello, how are you?" },
       { "role": "assistant", "content": "I am doing well, how can I help you today?" } ]`

    :return: a generator of delta string responses
    """
    try:
        async for chunk in await openai.ChatCompletion.acreate(
                model="gpt-3.5-turbo-0613",
                messages=messages,
                max_tokens=3000,
                stream=True
        ):
            content = chunk['choices'][0]['delta'].get('content', '')
            if content:
                yield content

    except OpenAIError as e:
        traceback.print_exc()
        yield f"EXCEPTION {str(e)}"
    except Exception as e:
        yield f"EXCEPTION {str(e)}"
        
def create_conversation_starter(user_prompt: str) -> List[Dict[str, str]]:
    """
    Given a user prompt, create a conversation history with the following format:
    `[ { "role": "user", "content": user_prompt } ]`

    :param user_prompt: a user prompt string
    :return: a conversation history
    """
    return [{"role": "user", "content": user_prompt}]

async def run_conversation(messages: List[Dict[str, str]], message_placeholder: DeltaGenerator) \
        -> List[Dict[str, str]]:
    full_response = ""
    message_placeholder.markdown("Thinking...")
    chunks = converse(messages)
    chunk = await anext(chunks, "END OF CHAT")
    while chunk != "END OF CHAT":
        print(f"Received chunk from LLM service: {chunk}")
        if chunk.startswith("EXCEPTION"):
            full_response = ":red[We are having trouble generating advice.  Please wait a minute and try again.]"
            break
        full_response = full_response + chunk
        message_placeholder.markdown(full_response + "▌")
        chunk = await anext(chunks, "END OF CHAT")
    message_placeholder.markdown(full_response)
    messages.append({"role": "assistant", "content": full_response})
    return messages


async def run_prompt(prompt: str,
                     message_placeholder: DeltaGenerator) \
        -> List[Dict[str, str]]:
    messages = create_conversation_starter(prompt)
    messages = await run_conversation(messages, message_placeholder)
    return messages

NameError: name 'DeltaGenerator' is not defined

In [31]:
content = converse([ { "role": "user", "content":prompt} ])

In [34]:
message_placeholder = []
messages = await run_conversation([ { "role": "user", "content":prompt} ], message_placeholder)
messages

NameError: name 'run_conversation' is not defined

In [74]:
def get_completion(prompt, model="gpt-3.5-turbo"):
	messages = [{"role": "user", "content": prompt}]

	response = openai.ChatCompletion.create(
		model=model,
		messages=messages,
		temperature=0,  # this is the degree of randomness of the model's output
	)
	return response.choices[0].message["content"]


def converse(prompt, messages=None, model="gpt-4", max_tokens=4000, temperature=0, top_p=1, frequency_penalty=0,
			 presence_penalty=0):
	# Add the user's message to the list of messages
	if messages is None:
		messages = []

	messages.append({"role": "user", "content": prompt})

	response = openai.ChatCompletion.create(
		model=model,
		messages=messages,
		temperature=temperature,
		max_tokens=max_tokens,
		top_p=top_p,
		frequency_penalty=frequency_penalty,
		presence_penalty=presence_penalty,
	).choices[0].message["content"]

	# Add the assistant's message to the list of messages
	messages.append({"role": "assistant", "content": response})

	return response, messages

In [86]:
message =[]
response, messages = converse(prompt, messages)
print(response)

Anomaly Analysis:

1. The patient's blood pressure seems to be fluctuating quite a bit, with systolic pressure ranging from 101 to 140 mmHg and diastolic pressure ranging from 62 to 90 mmHg. This is a concern as it indicates that the patient's blood pressure is not well controlled.

2. The patient's weight also shows significant fluctuations, ranging from 60.1 kg to 79.3 kg within a month. This could be due to measurement errors or it could indicate a health issue.

3. The patient's heart rate also varies significantly, ranging from 60 to 100 bpm. This could be due to physical activity, but if the high rates are occurring at rest, it could indicate a problem.

4. The patient's sleep duration varies from 5.4 to 8.9 hours. While this is within the normal range, the inconsistency could be affecting the patient's blood pressure control.

Key Insights:

1. The patient's blood pressure medication may need to be adjusted, as their blood pressure is not well controlled.

2. The patient's weigh

In [85]:
prompt = "The patient has a history of high blood pressure and has been taking medication for the past 3 months, please provide anomaly analysis and any key insights for the following health related data that can be provided to a doctor for smoother evaluation: \n"+df.to_json()

In [87]:
messages

[{'role': 'user',
  'content': 'The patient has a history of high blood pressure and has been taking medication for the past 3 months, please provide anomaly analysis and any key insights for the following health related data that can be provided to a doctor for smoother evaluation: \n{"Date":{"0":"2023-08-18","1":"2023-08-19","2":"2023-08-20","3":"2023-08-21","4":"2023-08-22","5":"2023-08-23","6":"2023-08-24","7":"2023-08-25","8":"2023-08-26","9":"2023-08-27","10":"2023-08-28","11":"2023-08-29","12":"2023-08-30","13":"2023-08-31","14":"2023-09-01","15":"2023-09-02","16":"2023-09-03","17":"2023-09-04","18":"2023-09-05","19":"2023-09-06","20":"2023-09-07","21":"2023-09-08","22":"2023-09-09","23":"2023-09-10","24":"2023-09-11","25":"2023-09-12","26":"2023-09-13","27":"2023-09-14","28":"2023-09-15","29":"2023-09-16"},"Steps":{"0":9372,"1":7722,"2":6741,"3":6382,"4":8082,"5":5023,"6":10112,"7":8543,"8":10779,"9":10756,"10":12867,"11":8259,"12":6727,"13":8809,"14":14619,"15":9703,"16":8724,